# Optimize an Electrode

In [3]:
recon_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/Richardson_DBS/Reco_Chron_Pain/VCVS_VPL/ea_reconstruction.mat'
target_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/Richardson_DBS/optimized_program/target_maps/flipped_SEEG_pain_intensity_vs_sham_inverted_pcc_inverse_r_map.nii'
out_dir = '/Users/cu135/hires_backdrops/test_electrode'

In [4]:
from stim_pyper.processing_utils.run_optimizer import OptimizerProcessor
optimizer = OptimizerProcessor(electrode_data_path=recon_path, nifti_path=target_path, output_path=out_dir)
d = optimizer.run()

[[ -8.82046   3.9988  -10.652  ]
 [ -9.60786   4.71071  -7.8736 ]
 [-10.2009    5.51964  -8.43272]
 [-10.6775    4.46854  -8.31959]
 [-10.8192    5.54901  -5.47567]
 [-11.4234    6.33489  -6.02034]
 [-11.9146    5.29093  -5.892  ]
 [-12.6139    6.48515  -3.36506]] Medtronic B33015
[[ 9.01359  4.57376 -8.98243]
 [10.0677   4.62635 -6.30857]
 [10.9469   5.20184 -6.81454]
 [ 9.89855  5.71787 -6.54161]
 [11.2915   5.25553 -3.92718]
 [12.1843   5.80132 -4.44139]
 [11.1093   6.34363 -4.16975]
 [12.8416   6.42344 -1.75197]] Medtronic B33015
[[ 15.1738   -19.5311    -5.12423 ]
 [ 16.2776   -20.5246    -2.29723 ]
 [ 16.5688   -19.3347    -2.27399 ]
 [ 15.4425   -19.8837    -2.05312 ]
 [ 17.1164   -20.939      0.635598]
 [ 17.4146   -19.7412     0.6531  ]
 [ 16.2727   -20.2932     0.884445]
 [ 17.7488   -20.7715     3.75606 ]] Medtronic B33015
All files appear to have the same voxel sizes: (np.float32(0.2), np.float32(0.2), np.float32(0.2)). No reslicing needed.
All files appear to have the same